In [1]:
import pandas as pd
import numpy as np
import math
from math import cos, sin, pi
import matplotlib.pyplot as plt

##Preprocessing - joining epac and atlc

In [23]:
df1 = pd.read_fwf('./data-input/ebtrk_atlc_1988_2018.txt', header = None)

In [24]:
df2 = pd.read_fwf('./data-input/ebtrk_epac_2001_2018.txt', header = None)

In [25]:
df1[12] = df1[12].apply(lambda x: x.replace(' ', '0'))
df1[12] = df1[12].str.zfill(12)

In [26]:
df1['NE_34'] = df1[12].str[:3]
df1['SE_34'] = df1[12].str[3:6]
df1['SW_34'] = df1[12].str[6:9]
df1['NW_34'] = df1[12].str[9:]
df1['max_34'] = df1[['NE_34', 'SE_34', 'SW_34', 'NW_34']].max(axis=1)
#df1['max_34_1'] = df.groupby([1])['max_34'].shift(-1)

In [27]:
for i , j in zip([12,13,14], [34,50,64]):
                 df1[i] = df1[i].apply(lambda x: x.replace(' ', '0'))
                 df1[i] = df1[i].str.zfill(12)
                 #df1['len'+str(j)] = df1[i].str.len()
                 df1['NE_'+str(j)] = df1[i].str[:3]
                 df1['SE_'+str(j)] = df1[i].str[3:6]   
                 df1['SW_'+str(j)] = df1[i].str[6:9]
                 df1['NW_'+str(j)] = df1[i].str[9:]
                 df1['max_'+str(j)] = df1[['NE_'+str(j), 'SE_'+str(j), 'SW_'+str(j), 'NW_'+str(j)]].max(axis=1)
                 
                 

In [28]:
df1

,0,1,2,3,4,5,6,7,8,9,...,NE_50,SE_50,SW_50,NW_50,max_50,NE_64,SE_64,SW_64,NW_64,max_64
0,AL0188,ALBERTO,80518,1988,32.0,77.5,20,1015,-99,-99,...,000,000,000,000,0.0,000,000,000,000,0.0
1,AL0188,ALBERTO,80600,1988,32.8,76.2,20,1014,-99,-99,...,000,000,000,000,0.0,000,000,000,000,0.0
2,AL0188,ALBERTO,80606,1988,34.0,75.2,20,1013,-99,-99,...,000,000,000,000,0.0,000,000,000,000,0.0
3,AL0188,ALBERTO,80612,1988,35.2,74.6,25,1012,-99,-99,...,000,000,000,000,0.0,000,000,000,000,0.0
4,AL0188,ALBERTO,80618,1988,37.0,73.5,25,1011,-99,-99,...,000,000,000,000,0.0,000,000,000,000,0.0
5,AL0188,ALBERTO,80700,1988,38.7,72.4,25,1009,-99,-99,...,000,000,000,000,0.0,000,000,000,000,0.0
6,AL0188,ALBERTO,80706,1988,40.0,70.8,30,1006,-99,-99,...,000,000,000,000,0.0,000,000,000,000,0.0
7,AL0188,ALBERTO,80712,1988,41.5,69.0,35,1002,-99,-99,...,000,000,000,000,0.0,000,000,000,000,0.0
8,AL0188,ALBERTO,80718,1988,43.0,67.5,35,1002,-99,-99,...,000,000,000,000,0.0,000,000,000,000,0.0
9,AL0188,ALBERTO,80800,1988,45.0,65.5,35,1004,-99,-99,...,000,000,000,000,0.0,000,000,000,000,0.0


In [29]:
latitude (deg N), longitude (deg W), maximum wind
speed (kt), minimum central pressure (hPa), radius of maximum wind
speed (nm), eye diameter (nm), pressure of the outer closed isobar (hPa),
radius of the outer closed isobar (nm), radii (nm) of 34 kt wind to the
NE, SE, SW and NW of the storm center, radii (nm) of 50 kt wind to the
NE, SE, SW and NW, radii (nm) of 64 kt wind to the NE, SE, SW, NW, and
a storm type code

SyntaxError: invalid syntax (<ipython-input-29-811811ab6679>, line 1)

In [30]:
df2[12] = df2[12].apply(lambda x: x.replace(' ', '0'))
df2[12] = df2[12].str.zfill(12)
#df2['len34'] = df2[12].str.len()
df2['NE_34'] = df2[12].str[:3]
df2['SE_34'] = df2[12].str[3:6]   
df2['SW_34'] = df2[12].str[6:9]
df2['NW_34'] = df2[12].str[9:]
df2['max_34'] = df2[['NE_34', 'SE_34', 'SW_34', 'NW_34']].max(axis=1)
                 

In [31]:
df2['NE_50']=df2[13]
df2['SE_50']=df2[14]
df2['SW_50']=df2[15]
df2['NW_50']=df2[16]
df2['NE_64']=df2[13]
df2['SE_64']=df2[14]
df2['SW_64']=df2[15]
df2['NW_64']=df2[16]
df2['max_50'] = df2[['NE_50', 'SE_50', 'SW_50', 'NW_50']].max(axis=1)
df2['max_64'] = df2[['NE_64', 'SE_64', 'SW_64', 'NW_64']].max(axis=1)
				


In [32]:
df1 = df1.drop([12,13,14], axis=1)
df2 = df2.drop([12,13,14,15,16,17,18,19,20], axis=1)

In [33]:
df2 = df2.rename(columns={ 0:'id', 1: 'name', 2: 'time', 3: 'year', 4: 'lat', 5:'long', 6:'m_w_speed', 
                    7: 'mcp', 8:'rmw', 9:'ed', 10:'press', 11:'r_press', 21:'type', 22:'code' }) 

In [34]:
df2

,id,name,time,year,lat,long,m_w_speed,mcp,rmw,ed,...,NE_50,SE_50,SW_50,NW_50,NE_64,SE_64,SW_64,NW_64,max_50,max_64
0,EP0101,ADOLPH,52518,2001,13.6,101.1,25,1006,-99,-99,...,0,0,0,0,0,0,0,0,0,0
1,EP0101,ADOLPH,52600,2001,13.7,101.2,30,1006,60,-99,...,0,0,0,0,0,0,0,0,0,0
2,EP0101,ADOLPH,52606,2001,13.8,101.2,30,1006,60,-99,...,0,0,0,0,0,0,0,0,0,0
3,EP0101,ADOLPH,52612,2001,13.7,100.9,30,1006,60,-99,...,0,0,0,0,0,0,0,0,0,0
4,EP0101,ADOLPH,52618,2001,13.6,100.5,35,1004,60,-99,...,0,0,0,0,0,0,0,0,0,0
5,EP0101,ADOLPH,52700,2001,13.5,100.2,40,1000,40,-99,...,0,0,0,0,0,0,0,0,0,0
6,EP0101,ADOLPH,52706,2001,13.3,100.0,50,997,40,-99,...,50,50,50,50,50,50,50,50,50,50
7,EP0101,ADOLPH,52712,2001,13.2,99.7,60,992,25,-99,...,50,50,50,50,50,50,50,50,50,50
8,EP0101,ADOLPH,52718,2001,13.5,99.5,75,980,25,-99,...,50,50,50,50,50,50,50,50,50,50
9,EP0101,ADOLPH,52800,2001,13.9,99.8,95,965,20,20,...,50,50,50,50,50,50,50,50,50,50


In [35]:
df1 = df1.rename(columns={ 0:'id', 1: 'name', 2: 'time', 3: 'year', 4: 'lat', 5:'long', 6:'m_w_speed', 
                    7: 'mcp', 8:'rmw', 9:'ed', 10:'press', 11:'r_press', 15:'type', 16:'code' }) 

In [36]:
df1

,id,name,time,year,lat,long,m_w_speed,mcp,rmw,ed,...,NE_50,SE_50,SW_50,NW_50,max_50,NE_64,SE_64,SW_64,NW_64,max_64
0,AL0188,ALBERTO,80518,1988,32.0,77.5,20,1015,-99,-99,...,000,000,000,000,0.0,000,000,000,000,0.0
1,AL0188,ALBERTO,80600,1988,32.8,76.2,20,1014,-99,-99,...,000,000,000,000,0.0,000,000,000,000,0.0
2,AL0188,ALBERTO,80606,1988,34.0,75.2,20,1013,-99,-99,...,000,000,000,000,0.0,000,000,000,000,0.0
3,AL0188,ALBERTO,80612,1988,35.2,74.6,25,1012,-99,-99,...,000,000,000,000,0.0,000,000,000,000,0.0
4,AL0188,ALBERTO,80618,1988,37.0,73.5,25,1011,-99,-99,...,000,000,000,000,0.0,000,000,000,000,0.0
5,AL0188,ALBERTO,80700,1988,38.7,72.4,25,1009,-99,-99,...,000,000,000,000,0.0,000,000,000,000,0.0
6,AL0188,ALBERTO,80706,1988,40.0,70.8,30,1006,-99,-99,...,000,000,000,000,0.0,000,000,000,000,0.0
7,AL0188,ALBERTO,80712,1988,41.5,69.0,35,1002,-99,-99,...,000,000,000,000,0.0,000,000,000,000,0.0
8,AL0188,ALBERTO,80718,1988,43.0,67.5,35,1002,-99,-99,...,000,000,000,000,0.0,000,000,000,000,0.0
9,AL0188,ALBERTO,80800,1988,45.0,65.5,35,1004,-99,-99,...,000,000,000,000,0.0,000,000,000,000,0.0


In [37]:
df = pd.concat([df1, df2])

/Users/anurkayeva/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [38]:
df

,NE_34,NE_50,NE_64,NW_34,NW_50,NW_64,SE_34,SE_50,SE_64,SW_34,...,max_50,max_64,mcp,name,press,r_press,rmw,time,type,year
0,000,000,000,000,000,000,000,000,000,000,...,0.0,0.0,1015,ALBERTO,-99,-99,-99,80518,*,1988
1,000,000,000,000,000,000,000,000,000,000,...,0.0,0.0,1014,ALBERTO,-99,-99,-99,80600,*,1988
2,000,000,000,000,000,000,000,000,000,000,...,0.0,0.0,1013,ALBERTO,-99,-99,-99,80606,*,1988
3,000,000,000,000,000,000,000,000,000,000,...,0.0,0.0,1012,ALBERTO,-99,-99,-99,80612,*,1988
4,000,000,000,000,000,000,000,000,000,000,...,0.0,0.0,1011,ALBERTO,-99,-99,-99,80618,*,1988
5,000,000,000,000,000,000,000,000,000,000,...,0.0,0.0,1009,ALBERTO,-99,-99,-99,80700,*,1988
6,000,000,000,000,000,000,000,000,000,000,...,0.0,0.0,1006,ALBERTO,1012,90,-99,80706,*,1988
7,100,000,000,050,000,000,100,000,000,050,...,0.0,0.0,1002,ALBERTO,1012,60,-99,80712,*,1988
8,100,000,000,050,000,000,100,000,000,050,...,0.0,0.0,1002,ALBERTO,1008,50,-99,80718,*,1988
9,-99,000,000,-99,000,000,-99,000,000,-99,...,0.0,0.0,1004,ALBERTO,1008,50,-99,80800,*,1988


###Preprocess joining with tropical

In [39]:
df_tracks = df[['name','year']]

In [40]:
df_tracks = df_tracks.drop_duplicates()
df_tracks['in_track'] = True

In [44]:
df_tropical = pd.read_csv('./data-input/TCE-DAT_2015-exposure_1950-2015.csv')

In [53]:
df_tropical['in_tropical'] = True

In [46]:
df_tropical[['name','year', 'in_tropical']] = df_tropical[['TC_name','year', 'in_tropical']]
df_tropical =df_tropical[['name','year']]
df_tropical = df_tropical.drop_duplicates()

In [47]:
df_tropical

,name,year
0,UNNAMED,1950
1,ELSIE,1950
5,GRACE,1950
19,ABLE,1950
21,BAKER,1950
36,DOG,1950
47,KEZIA,1950
50,MISSATHA,1950
51,OSSIA,1950
55,GEORGE,1950


In [54]:
result = pd.merge(df_tracks, df_tropical, how='outer', on=['name', 'year'])

In [55]:
result

,name,year,in_track,in_tropical
0,ALBERTO,1988,True,True
1,BERYL,1988,True,True
2,CHRIS,1988,True,True
3,DEBBY,1988,True,True
4,ERNESTO,1988,True,NaN
5,NOT NAM,1988,True,NaN
6,FLORENC,1988,True,NaN
7,GILBERT,1988,True,True
8,HELENE,1988,True,NaN
9,ISAAC,1988,True,True


In [56]:
result[result['in_track']!=True][result['in_tropical']==True][result['name']!='UNNAMED']

/Users/anurkayeva/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,name,year,in_track,in_tropical
725,ELSIE,1950,NaN,True
726,GRACE,1950,NaN,True
727,ABLE,1950,NaN,True
728,BAKER,1950,NaN,True
729,DOG,1950,NaN,True
730,KEZIA,1950,NaN,True
731,MISSATHA,1950,NaN,True
732,OSSIA,1950,NaN,True
733,GEORGE,1950,NaN,True
734,HOW,1950,NaN,True
